In [1]:
from openapi_client import openapi
from openapi_genclient.models.candle_resolution import CandleResolution

import pandas as pd, numpy as np
import numpy as np
from scipy import stats as sts
import time
from time import sleep
import datetime
from datetime import datetime, timedelta, timezone
from pytz import timezone
import pytz
from dateutil.tz import tzutc

# Teleg
import requests

import schedule

# Rotate proxies
from lxml.html import fromstring
from itertools import cycle
import traceback

# tqdm
from tqdm import tqdm

In [2]:
# Send telegram info     
def send_telegram(text, proxies={}):
    token = 'tok_tlg'    
    url = 'https://api.telegram.org/bot'
    channel_id = '@channel_'
    url += token
    method = url + '/sendMessage'
   
    r = requests.post(        
            method, 
            data={
                'chat_id': channel_id,
                'text': text
                },
#            proxies=proxies
        )

    if r.status_code != 200:
        raise Exception('post_text error')

def f_teleg_send_msg(bot_msg):
    str_status = ''    
    try:
        print('Telegram data sending...')
        send_telegram(bot_msg)
        str_status = 'Telegram data send - Done.'
    except:        
        str_status = 'Skipping. Connnection error'            
    return str_status

In [3]:
# send_telegram('Test_msg_from_nicholas_id')

In [5]:
# Define confidence intervals
def f_mean_hi_lo(df_candles_usd):
    a1 = 0.999
    N = df_candles_usd.shape[0]
    f = N - 1
    B = np.std(df_candles_usd) / np.sqrt(N)
    
    Xm = np.average(df_candles_usd)
    
    if N > 1:
        LCLx = Xm - sts.t.ppf(a1, f) * B # More large int
        UCLx = Xm + sts.t.ppf(a1, f) * B # More large int

        s_valid = df_candles_usd.between(LCLx, UCLx)
    #     print(t_valid)

        t = (df_candles_usd - Xm) / np.std(df_candles_usd)
        t_valid = np.abs(t) <= 2.08 # t_cr = 2.08
    #     print(t_valid)

        integral_valid = np.logical_and(s_valid, t_valid)
        
        # mean_hi_lo = np.mean(df_candles_usd[integral_valid])
        mean_hi_lo = np.mean(df_candles_usd[s_valid])
    else:
        mean_hi_lo = Xm
    print('Mean calculate Done.')
    return  mean_hi_lo #integral_valid, s_valid, t_valid, LCLx, Xm, UCLx

In [7]:
# Get Candles Info
def f_candles_get(apiMrkt, df_inst, dt_now, delta_min, candles_resol = CandleResolution._5MIN):    
    
    # Add columns time/date:
    f_lm_time = lambda x: [x['time'][i].strftime('%H:%M:%S')
                           for i in range(x['time'].shape[0])]
    
    # Full df fro stocks
    df_candles_full = pd.DataFrame()
    
    try:
        for i in (range(df_inst.shape[0])):
                   
            # Get Candle -> timedelta: minutes, hours, weeks
#             DS_ = (dt_now - timedelta(minutes = delta_min)).strftime('%Y-%m-%d %H:%M:%S')
            DS_ = (dt_now).strftime('%Y-%m-%d') + str(' 07:00:00') # 07:00:00
            DS = datetime.strptime(DS_ + ' GMT', '%Y-%m-%d %H:%M:%S %Z').astimezone().isoformat()            
            
            DE_ = (dt_now).strftime('%Y-%m-%d %H:%M:%S')
            DE = datetime.strptime(DE_ + ' GMT', '%Y-%m-%d %H:%M:%S %Z').astimezone().isoformat()
            print(DS)
            print(DE)
            
            td = (datetime.strptime(DE_, '%Y-%m-%d %H:%M:%S') - datetime.strptime(DS_, '%Y-%m-%d %H:%M:%S'))
            
            try:
                thread = apiMrkt.market.market_candles_get(df_inst['figi'][i], #df_inst['figi'][i], 'BBG0013HGFT4'
                                                           DS, DE,
                                                           candles_resol, # CandleResolution._5MIN,
                                                           _preload_content = True,
        #                                                    _request_timeout,
                                                           async_req=True
                                                          )
                res_candles_ = thread.get()
                res_candles = res_candles_.payload           
                
                df_candles = pd.DataFrame()
                f_candles = lambda x: [df_candles.append(res_candles.to_dict()['candles'][x], ignore_index=True)
                                       for x in range(len(res_candles.candles))]

                df_candles_ = f_candles(777)

                if len(df_candles_) != 0:
                    df_candles_full = df_candles_full.append(df_candles_, ignore_index = True)
                
                # Clear mem
                del df_candles_, df_candles
            
            except Exception as exc:
                print('Exception thread: ' + str(exc)) #inst:
                continue
                
        if (df_candles_full.shape[0]) > 0:
            ts_ = DS[11:-6]
            te_ = DE[11:-6]

            df_candles_usd=\
            df_candles_full.assign(time2 = f_lm_time(df_candles_full.reset_index(drop=True)))

            dts_candles = df_candles_full['time'][0].isoformat()[11:]

            df_candles_usd_in=\
            df_candles_usd[df_candles_usd['time2']
                          .between(ts_, te_)
                          ]

            Xm_every5min = (f_mean_hi_lo(df_candles_usd_in['h']) +
                            f_mean_hi_lo(df_candles_usd_in['l'])
                           )/2
            
            bot_msg = ('Time * '+ dts_candles +'\n'+\
                       'Lag -> '+ str(td.total_seconds()//3600)[:-2] +' h ' +\
                                str((td.total_seconds() % 3600)//60)[:-2] +' min'+ '\n' + \
                       'Mean ≈ ' + str(round(Xm_every5min,4))
                      )
            
            
        else:
            bot_msg = ''
            print('df_candles_full.shape[0] == 0')
            
    except Exception as exc:
            bot_msg = ''
            print('Exception f_candles_get: ' + str(exc)) #inst            
            pass
    
    return bot_msg #df_candles_full

In [10]:
# sandbox Nick
token = 'tok_trade'
apiMrkt = openapi.sandbox_api_client(token)

thread = apiMrkt.market.market_currencies_get(async_req=True,
                                          _preload_content = True)
res_instr_ = thread.get()
res_instr = res_instr_.payload

# Get instrument
df_inst = pd.DataFrame()
f_inst = lambda x: [df_inst.append((res_instr.to_dict()['instruments'][x]), ignore_index=True) 
               for x in range(len(res_instr.instruments))] 

df_inst_ = f_inst(777)
df_inst = df_inst.append(df_inst_, ignore_index = True)[0:1]
print('Instruments Done.')

# Get Candles
def job():
    # Bot_msg
    print('Job executing ...')
    dt_now = datetime.now()
    delta_min = 5
    candles_resol = CandleResolution._5MIN
    
    bot_msg = f_candles_get(apiMrkt, df_inst[0:1],
                            dt_now, delta_min,
                            candles_resol = candles_resol
                           )
    
    # ##### Telegram section #####
    if bot_msg != '':
        str_status = f_teleg_send_msg(bot_msg)
    else:
        #         print('bot_msg Empty.')
        str_status = 'bot_msg Empty.'
    print(str_status + ' ' + str(datetime.now()))

def schedule_every_five_minutes():
    job() #for the first job to run
#     schedule.every(300).seconds.do(job) # 5min = 5*60 sec.
    time.sleep(60)
    return schedule.CancelJob

if __name__ == '__main__':
    
    for i in range(1,19,1):
        td = (datetime.strptime('12:35:00', '%H:%M:%S') + timedelta(minutes = (i*5))).time()
#         print(str(td))
        schedule.every().monday.at(str(td)).do(job)
        schedule.every().tuesday.at(str(td)).do(job)
        schedule.every().wednesday.at(str(td)).do(job)
        schedule.every().thursday.at(str(td)).do(job)
        schedule.every().friday.at(str(td)).do(job)
    
    
    try:
        while True:
            if (datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f') > \
                   (datetime.now().date().strftime('%Y-%m-%d') + ' 04:00:00.000000'))&\
               (datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f') < \
                   (datetime.now().date().strftime('%Y-%m-%d') + ' 07:00:00.000000'))&\
               (datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f') > \
                   (datetime.now().date().strftime('%Y-%m-%d') + ' 23:40:00.000000')):
                schedule.cancel_job(job)
                print('cancel_job')
            else:
                schedule.run_pending()
                time.sleep(1)
                
    except Exception as exc:   
        print('Exception main: ' + str(exc)) #inst
#         print('Exception traceback:\n', traceback.format_exc())
        pass